# Global Daily Event Analysis: Marine Heatwave ID & Tracking using `MarEx`

### `MarEx` Processing Pipeline for Gridded Datasets:

1. **Morphological Pre-Processing**
    - Performs binary morphological closing using `dask_image.ndmorph` to fill small spatial holes up to `R_fill` cells in radius 
    - Executes binary opening to remove isolated small features of order `R_fill`
    - Fills gaps in time to maintain event continuity for interruptions up to `T_fill` time steps
    - Filters out smallest objects below the `area_filter_quartile` percentile threshold

2. **Blob Identification**
    - Labels spatially connected components using efficient connected-component algorithm in `dask_image.ndmeasure`
    - Computes blob properties (area, centroid, boundaries)

3. **Temporal Tracking**
    - Identifies blob overlaps between consecutive time frames
    - Connects objects across time, applying the following criteria for splitting, merging, & persistence:
        - Connected objects must overlap by at least fraction `overlap_threshold` of the smaller area
        - Merged objects retain their original ID, but partition the child area based on the parent of the _nearest-neighbour_ cell (or centroid distance)

4. **Graph Reduction & Finalisation**
    - Constructs the complete temporal graph of object evolution through time
    - Resolves object connectivity graph using `scipy.sparse.csgraph.connected_components`
    - Creates globally unique IDs for each tracked extreme event
    - Maps objects into efficient ID-time space for convenient analysis
    - Computes comprehensive statistics about the lifecycle of each event

The pipeline leverages **dask** for distributed parallel computation, enabling efficient processing of large datasets. \
A 40-year global daily analysis at 0.25° resolution on 32 cores takes
- Basic (i.e. Scannell et al., which involves no merge/split criteria or tracking):  ~5 minutes
- Full Split/Merge Thresholding & Merge Tracking:  ~40 minutes

In [1]:
import xarray as xr
import dask
from getpass import getuser
from pathlib import Path

import marEx
import marEx.helper as hpc

In [2]:
# Lustre Scratch Directory
scratch_dir = Path('/scratch') / getuser()[0] / getuser()

In [3]:
# Start Dask Cluster
client = hpc.start_local_cluster(n_workers=32, threads_per_worker=1,
                                 scratch_dir = scratch_dir / 'clients')  # Specify temporary scratch directory for dask to use

Dask Scratch: '/scratch/b/b382615/clients/tmp29zfqzyy'
Memory per Worker: 15.74 GB


/home/b/b382615/opt/anaconda3/lib/python3.10/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46619 instead
  warnings.warn(


Hostname: l40038
Forward Port: l40038:46619
Dashboard Link: localhost:46619/status


In [ ]:
# Choose optimal chunk size & load data
#   N.B.: This is crucial for dask (not only for performance, but also to make the problem tractable)
#         The operations are eventually global-in-space, and so requires the spatial dimension to be contiguous/unchunked
#         We can adjust the chunk size in time depending on available system memory.

chunk_size = {'time': 25, 'lat': -1, 'lon': -1}

In [5]:
# Load Pre-processed Data (cf. `01_preprocess_extremes.ipynb`)

file_name = scratch_dir / 'mhws' / 'extremes_binary_gridded.zarr'
ds = xr.open_zarr(str(file_name), chunks=chunk_size)

In [ ]:
# Run ID, Tracking, & Merging

tracker = marEx.tracker(ds.extreme_events, 
                       ds.mask.where((ds.lat < 85) &
                                     (ds.lat > -90), other=False),  # Modify Mask: Anisotropy of the lat/lon grid near the poles biases the ID & Tracking
                       area_filter_quartile = 0.5,                  # Remove the smallest 50% of the identified coherent extreme areas
                       R_fill = 8,                                  # Fill small holes with radius < 8 _cells_
                       T_fill = 2,                                  # Allow gaps of 2 days and still continue the event tracking with the same ID
                       allow_merging = True,                        # Allow extreme events to split/merge. Keeps track of merge events & unique IDs.
                       overlap_threshold = 0.5,                     # Overlap threshold for merging events. If overlap < threshold, events keep independent IDs.
                       nn_partitioning = True,                      # Use new NN method to partition merged children areas. If False, reverts to old method of Di Sun et al. 2023.
                       verbosity=1)                                 # Choose Verbosity Level (0=None, 1=Basic, 2=Advanced/Timing)

extreme_events_ds, merges_ds = tracker.run(return_merges=True)
extreme_events_ds

Finished filling spatial holes
Finished filling spatio-temporal holes
Finished filtering small objects
Finished object identification
Finished calculating object properties
Finished finding overlapping objects


/home/b/b382615/opt/anaconda3/lib/python3.10/site-packages/dask/array/core.py:5092: PerformanceWarning: Increasing number of chunks by factor of 11
  result = blockwise(


In [7]:
merges_ds

In [8]:
# Save IDed/Tracked/Merged Events to `zarr` for more efficient parallel I/O

file_name = scratch_dir / 'mhws' / 'extreme_events_merged_gridded.zarr'
extreme_events_ds.to_zarr(file_name, mode='w')

### Run Basic Tracking for Comparison
N.B.: This is the current standard method used in the literature, which involves _No_ temporal gap filling, _No_ merging/splitting and _No_ independent event tracking.

In [9]:
# Run Basic Tracking

tracker = marEx.tracker(ds.extreme_events, 
                       ds.mask.where((ds.lat < 85) &
                                     (ds.lat > -90), other=False),  # Modify Mask: Anisotropy of the lat/lon grid near the poles biases the ID & Tracking
                       area_filter_quartile = 0.5,                  # Remove the smallest 50% of the identified coherent extreme areas
                       R_fill = 8,                                  # Fill small holes with radius < 8 _cells_
                       T_fill = 0,                                  # No temporal hole filling
                       allow_merging = False,                       # Do not allow extreme events to split/merge. All touching events adopt the same ID forever (after _and_ before (!)).
                       verbosity=1)

extreme_events_basic_ds = tracker.run()
extreme_events_basic_ds

Finished filling spatial holes
Finished filling spatio-temporal holes
Finished filtering small objects
Finished tracking all extreme events!


Tracking Statistics:
   Binary Hobday to Processed Area Fraction: 0.8525119484330325
   Total Object Area IDed (cells): 361690641.0
   Number of Initial Pre-Filtered Objects: 210966
   Number of Final Filtered Objects: 105545
   Area Cutoff Threshold (cells): 651
   Accepted Area Fraction: 0.9064810029187347
   Total Events Tracked: 10190




<xarray.Dataset> Size: 33GB
Dimensions:   (lat: 721, lon: 1440, time: 8036)
Coordinates:
  * lat       (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon       (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time      (time) datetime64[ns] 64kB 2020-01-01 2020-01-02 ... 2041-12-31
Data variables:
    ID_field  (time, lat, lon) int32 33GB dask.array<chunksize=(25, 721, 1440), meta=np.ndarray>
Attributes:
    allow_merging:               0
    N_objects_prefiltered:       210966
    N_objects_filtered:          105545
    N_events_final:              10190
    R_fill:                      8
    T_fill:                      0
    area_filter_quartile:        0.5
    area_threshold (cells):      651.0
    accepted_area_fraction:      0.9064810029187347
    preprocessed_area_fraction:  0.8525119484330325

In [10]:
# Save IDed Events to `zarr` for more efficient parallel I/O

file_name = scratch_dir / 'mhws' / 'extreme_events_basic_gridded.zarr'
extreme_events_basic_ds.to_zarr(file_name, mode='w')